In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from copy import deepcopy
from collections import deque
import io
from PIL import Image
import imageio

do_record = True         
record_interval = 2      
frames = []             
frame_counter = 0

do_record_phase2 = True 
phase2_frames = []      
phase2_frame_counter = 0

max_attempts = 500       
TARGET_DOORS = 3         # 3 doors puzzles
TARGET_RARE = 5          # 5 rare items

tiles = ["H", "V", "NE", "SE", "SW", "NW"]

connections = {
    "H":  (False, True,  False, True),
    "V":  (True,  False, True,  False),
    "NE": (True,  True,  False, False),
    "SE": (False, True,  True,  False),
    "SW": (False, False, True,  True),
    "NW": (True,  False, False, True)
}

directions = {
    "up":    (0, -1, 0, 2),
    "right": (1, 0, 1, 3),
    "down":  (0, 1, 2, 0),
    "left":  (-1, 0, 3, 1)
}

tile_colors = {
    "H": "lightblue",
    "V": "lightgreen",
    "NE": "lightyellow",
    "SE": "lightpink",
    "SW": "lightgray",
    "NW": "lightcoral"
}
uncollapsed_color = "#dddddd"

def initialize_grid(width, height, tileset):
    """Return a grid where each cell starts with the full set of tiles."""
    return [[set(tileset) for _ in range(width)] for _ in range(height)]

def find_lowest_entropy_cell(grid):
    """Return (x, y) of a cell with >1 possibility and minimal count."""
    min_e = None
    candidates = []
    for y in range(len(grid)):
        for x in range(len(grid[0])):
            poss = grid[y][x]
            if len(poss) > 1:
                e = len(poss)
                if min_e is None or e < min_e:
                    min_e = e
                    candidates = [(x, y)]
                elif e == min_e:
                    candidates.append((x, y))
    return random.choice(candidates) if candidates else None

def propagate(grid, x, y):
    """Propagate constraints from cell (x, y) to its neighbors. Return True if success; False on contradiction."""
    height = len(grid)
    width = len(grid[0])
    queue = [(x, y)]
    while queue:
        cx, cy = queue.pop(0)
        current_set = grid[cy][cx]
        for d, (dx, dy, c_idx, n_idx) in directions.items():
            nx, ny = cx + dx, cy + dy
            if 0 <= nx < width and 0 <= ny < height:
                neighbor_set = grid[ny][nx]
                new_set = set()
                for candidate in neighbor_set:
                    if any(connections[t][c_idx] == connections[candidate][n_idx] for t in current_set):
                        new_set.add(candidate)
                if new_set != neighbor_set:
                    if not new_set:
                        return False
                    grid[ny][nx] = new_set
                    queue.append((nx, ny))
    return True

def is_grid_collapsed(grid):
    """Return True if every cell has exactly one possibility."""
    return all(len(cell)==1 for row in grid for cell in row)

def capture_frame(grid):
    """Record the current tile-grid state (Phase 1) for GIF creation."""
    global frames, frame_counter
    if not do_record:
        return
    if frame_counter % record_interval != 0:
        frame_counter += 1
        return
    height = len(grid)
    width = len(grid[0])
    fig, ax = plt.subplots(figsize=(width*1.3, height*1.3))
    ax.set_xlim(0, width)
    ax.set_ylim(0, height)
    ax.invert_yaxis()
    ax.set_xticks([])
    ax.set_yticks([])
    for y in range(height):
        for x in range(width):
            cell = grid[y][x]
            if len(cell)==1:
                t = list(cell)[0]
                color = tile_colors.get(t, "white")
                text = t
            else:
                color = uncollapsed_color
                text = str(len(cell))
            rect = patches.Rectangle((x, y), 1, 1, edgecolor="black", facecolor=color)
            ax.add_patch(rect)
            ax.text(x+0.5, y+0.5, text, ha="center", va="center", fontsize=8)
    ax.set_title(f"Phase 1 Step {frame_counter}")
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    plt.close(fig)
    buf.seek(0)
    frames.append(Image.open(buf))
    frame_counter += 1

def wfc_greedy(width, height):
    """
    Greedily collapse the tile grid.
    Force the entrance at top-left and exit at bottom-right to "V".
    If a contradiction occurs, return None.
    """
    grid = initialize_grid(width, height, tiles)
    grid[0][0] = {"V"}
    if not propagate(grid, 0, 0):
        return None
    grid[height-1][width-1] = {"V"}
    if not propagate(grid, width-1, height-1):
        return None
    while not is_grid_collapsed(grid):
        cell = find_lowest_entropy_cell(grid)
        if cell is None:
            break
        x, y = cell
        poss = list(grid[y][x])
        random.shuffle(poss)
        grid[y][x] = {poss[0]}
        if not propagate(grid, x, y):
            return None
        capture_frame(grid)
    return grid

def run_wfc_greedy(width, height, max_attempts=500):
    for _ in range(max_attempts):
        grid = wfc_greedy(width, height)
        if grid is not None:
            return grid
    return None

def convert_tiles_to_maze(tile_grid):
    """
    Convert the collapsed tile grid to an ASCII maze.
    Maze dimensions: (2*h + 1) rows x (2*w + 1) columns.
    Walls are '#' and corridors are '.'.
    """
    h = len(tile_grid)
    w = len(tile_grid[0])
    maze_h = 2 * h + 1
    maze_w = 2 * w + 1
    maze = [["#" for _ in range(maze_w)] for _ in range(maze_h)]
    for y in range(h):
        for x in range(w):
            tile = list(tile_grid[y][x])[0]
            up, right, down, left = connections[tile]
            cy = 2*y + 1
            cx = 2*x + 1
            maze[cy][cx] = "."
            if up:    maze[cy-1][cx] = "."
            if right: maze[cy][cx+1] = "."
            if down:  maze[cy+1][cx] = "."
            if left:  maze[cy][cx-1] = "."
    return maze

def force_entrance_exit(maze):
    """Force the top-left and bottom-right cells of the ASCII maze to be open."""
    rows = len(maze)
    cols = len(maze[0])
    maze[0][0] = "."
    maze[rows-1][cols-1] = "."
    return maze

def connect_disconnected_components(maze):
    """
    For each corridor ('.') that is not part of the main connected component (reachable from (0,0)),
    carve out one wall to attach it.
    """
    rows = len(maze)
    cols = len(maze[0])
    
    def bfs(start):
        visited = set()
        q = deque([start])
        visited.add(start)
        while q:
            r, c = q.popleft()
            for dr, dc in [(1,0), (-1,0), (0,1), (0,-1)]:
                nr, nc = r+dr, c+dc
                if 0<=nr<rows and 0<=nc<cols and maze[nr][nc]=="." and (nr, nc) not in visited:
                    visited.add((nr, nc))
                    q.append((nr, nc))
        return visited
    
    main_component = bfs((0,0))
    for r in range(rows):
        for c in range(cols):
            if maze[r][c]=="." and (r,c) not in main_component:
                for dr, dc in [(1,0), (-1,0), (0,1), (0,-1)]:
                    wr, wc = r+dr, c+dc
                    if 0<=wr<rows and 0<=wc<cols and maze[wr][wc]=="#":
                        # If the cell beyond the wall is in the main component carve the wall
                        or_, oc = wr+dr, wc+dc
                        if 0<=or_<rows and 0<=oc<cols and (or_, oc) in main_component:
                            maze[wr][wc] = "."
                            main_component = main_component.union(bfs((r,c)))
                            break
    return maze

def find_path_in_maze(maze, start=None, goal=None):
    """
    Extract a solution path via BFS from start to goal.
    By default, start is (0,0) and goal is (rows-1, cols-1).
    Returns a list of (r, c) coordinates along the path.
    """
    rows = len(maze)
    cols = len(maze[0])
    if start is None:
        start = (0, 0)
    if goal is None:
        goal = (rows-1, cols-1)
    q = deque([start])
    came_from = {start: None}
    found = None
    while q:
        current = q.popleft()
        if current == goal:
            found = current
            break
        r, c = current
        for dr, dc in [(1,0), (-1,0), (0,1), (0,-1)]:
            nr, nc = r+dr, c+dc
            if 0<=nr<rows and 0<=nc<cols and maze[nr][nc]=="." and (nr,nc) not in came_from:
                came_from[(nr,nc)] = current
                q.append((nr,nc))
    if found is None:
        return []
    path = []
    cur = found
    while cur is not None:
        path.append(cur)
        cur = came_from[cur]
    path.reverse()
    return path
    
# Hardcoded the placement tiles on which the wfc will work but can change it later on
def place_doors_and_keys(maze, path, num_doors=TARGET_DOORS):
    """
    Place exactly num_doors doors ("D") along the solution path at roughly 25%,50%,75% positions.
    For each door, place a key ("K") three cells earlier along the path.
    Only replace cells that are corridors (".").
    """
    L = len(path)
    if L < 6:
        return maze
    door_indices = [max(3, int(L*0.25)),
                    max(3, int(L*0.50)),
                    max(3, int(L*0.75))]
    door_indices = [i if i < L-1 else L-2 for i in door_indices]
    key_indices = [max(0, i-3) for i in door_indices]
    for idx in key_indices:
        r, c = path[idx]
        if maze[r][c] == ".":
            maze[r][c] = "K"
    for idx in door_indices:
        r, c = path[idx]
        if maze[r][c] == ".":
            maze[r][c] = "D"
    return maze

def place_rare_items(maze, path, num_rare=TARGET_RARE):
    """
    Place exactly num_rare rare items ("R") in corridor cells that are not on the solution path.
    Rare items are placed randomly among eligible cells.
    """
    rows = len(maze)
    cols = len(maze[0])
    path_set = set(path)
    candidates = []
    for r in range(rows):
        for c in range(cols):
            if maze[r][c]=="." and (r,c) not in path_set:
                candidates.append((r,c))
    if len(candidates) < num_rare:
        selected = random.sample(candidates, len(candidates))
    else:
        selected = random.sample(candidates, num_rare)
    for (r,c) in selected:
        maze[r][c] = "R"
    return maze


def print_tile_grid(grid):
    print("Tile Grid:")
    for row in grid:
        print(" ".join(list(cell)[0] for cell in row))

def print_ascii_maze(maze, title="ASCII Maze"):
    print(title)
    for line in maze:
        print("".join(line))

phase2_frames = []
phase2_frame_counter = 0
def capture_ascii_frame(maze, phase_label="Phase2"):
    global phase2_frames, phase2_frame_counter
    rows = len(maze)
    cols = len(maze[0])
    fig, ax = plt.subplots(figsize=(cols/3, rows/3))
    ax.set_xlim(0, cols)
    ax.set_ylim(0, rows)
    ax.set_aspect("equal")
    ax.invert_yaxis()
    ax.set_xticks([]); ax.set_yticks([])
    ax.set_title(f"{phase_label} Step {phase2_frame_counter}")
    for r in range(rows):
        for c in range(cols):
            ch = maze[r][c]
            if ch == "#":
                color = "dimgray"
            elif ch == ".":
                color = "white"
            elif ch == "D":
                color = "red"
            elif ch == "K":
                color = "gold"
            elif ch == "R":
                color = "purple"
            else:
                color = "white"
            rect = patches.Rectangle((c, r), 1, 1, edgecolor="black", facecolor=color)
            ax.add_patch(rect)
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    plt.close(fig)
    buf.seek(0)
    phase2_frames.append(Image.open(buf))
    phase2_frame_counter += 1

def display_maze(maze, solution_path=set(), title="Final Maze with Puzzle Elements"):
    plt.figure(figsize=(len(maze[0])/3, len(maze)/3))
    ax = plt.gca()
    ax.set_xlim(0, len(maze[0]))
    ax.set_ylim(0, len(maze))
    ax.set_aspect("equal")
    ax.invert_yaxis()
    ax.set_xticks([]); ax.set_yticks([])
    plt.title(title)
    for r in range(len(maze)):
        for c in range(len(maze[0])):
            ch = maze[r][c]
            if ch == "D":
                color = "red"
            elif ch == "K":
                color = "gold"
            elif ch == "R":
                color = "purple"
            elif (r,c) in solution_path:
                color = "lightgreen"
            elif ch == "#":
                color = "dimgray"
            elif ch == ".":
                color = "white"
            else:
                color = "white"
            rect = patches.Rectangle((c, r), 1, 1, edgecolor="black", facecolor=color)
            ax.add_patch(rect)
    plt.show()


if __name__=="__main__":
    random.seed(42)
    # Use a 5x5 tile grid to produce an 11x11 ASCII maze. Good for fast computation of mazes.
    grid_w, grid_h = 5, 5
    attempts = 0
    solution_grid = None
    ascii_maze_phase1 = None
    ascii_maze_connected = None
    final_ascii_maze = None
    sol_path = None
    # Phase 1: Generate tile grid using greedy WFC
    while attempts < max_attempts:
        attempts += 1
        grid = wfc_greedy(grid_w, grid_h)
        if grid is None:
            continue
        print_tile_grid(grid)
        maze_candidate = convert_tiles_to_maze(grid)
        maze_candidate = force_entrance_exit(maze_candidate)
        ascii_maze_phase1 = maze_candidate
        print_ascii_maze(ascii_maze_phase1, title="Phase 1: Initial ASCII Maze")
        display_maze(ascii_maze_phase1, solution_path=set(), title="Phase 1: Initial ASCII Maze")
        # Phase 1.5: Connector step: patch isolated corridors.
        maze_connected = connect_disconnected_components(deepcopy(maze_candidate))
        ascii_maze_connected = maze_connected
        capture_ascii_frame(ascii_maze_connected, phase_label="Connected Maze")
        print_ascii_maze(ascii_maze_connected, title="Phase 2: Connected ASCII Maze")
        display_maze(ascii_maze_connected, solution_path=set(), title="Phase 2: Connected ASCII Maze")
        sol_path = find_path_in_maze(ascii_maze_connected, start=(0,0), goal=(len(maze_connected)-1, len(maze_connected[0])-1))
        if sol_path and len(sol_path) >= 10:
            solution_grid = grid
            break
    if solution_grid is None:
        print("Failed to generate a valid maze with a connected solution within", max_attempts, "attempts.")
    else:
        print("Maze generated in", attempts, "attempt(s).")
        print_ascii_maze(ascii_maze_phase1, title="Phase 1: Final ASCII Maze (Before Connecting)")
        print_ascii_maze(ascii_maze_connected, title="Phase 2: Connected ASCII Maze (Before Puzzle Placement)")
        # Phase 2: Place puzzle elements.
        maze_with_puzzles = place_doors_and_keys(deepcopy(ascii_maze_connected), sol_path, num_doors=TARGET_DOORS)
        maze_with_puzzles = place_rare_items(maze_with_puzzles, sol_path, num_rare=TARGET_RARE)
        final_ascii_maze = maze_with_puzzles
        capture_ascii_frame(final_ascii_maze, phase_label="Final Puzzle Maze")
        print_ascii_maze(final_ascii_maze, title="Phase 3: Final ASCII Maze with Puzzle Elements")
        display_maze(final_ascii_maze, solution_path=set(sol_path),
                     title="Final Maze with Puzzle Elements & Connected Solution")
        
        if do_record and frames:
            phase1_gif = "phase1_1.gif"
            imageio.mimsave(phase1_gif, frames, duration=0.2)
            print("Phase 1 GIF saved as:", phase1_gif)
        if do_record and phase2_frames:
            phase2_gif = "phase2_1.gif"
            imageio.mimsave(phase2_gif, phase2_frames, duration=0.2)
            print("Phase 2 GIF saved as:", phase2_gif)
